In [1]:
import numpy as np
import os
import pandas as pd
import time
from itertools import product
import math

In [2]:
start = 500000

end = 750000


In [3]:
df_disease_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_whole.tsv")
df_drug_sim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/9_gold_drug_similarity_whole.tsv")
pos = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_pos.tsv")
#neg = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_neg.tsv")

print len(pos)
#print len(neg)

new_asso_li = list()
for index, row in pos.iterrows():
    query_drug = row['drug']
    query_disease = row['disease']
    query_asso = row['association']
    new_asso_li.append((query_drug, query_disease, query_asso))

new_pos_li = list()
for index2, row2 in pos.iterrows():
    pos_drug = row2['drug']
    pos_disease = row2['disease']
    new_pos_li.append((pos_drug, pos_disease)) 

print 'start : ' + time.strftime('%a %H:%M:%S')
desc_pair = list(product(new_asso_li, new_pos_li))
print 'end : ' + time.strftime('%a %H:%M:%S')

descriptor = pd.DataFrame()
cnt = 0
print 'start : ' + time.strftime('%a %H:%M:%S')
for pair in desc_pair[start:end]:
    query_drug = pair[0][0]
    query_disease = pair[0][1]
    query_asso = pair[0][2]
    pos_drug = pair[1][0]
    pos_disease = pair[1][1]
    if (query_drug == pos_drug) & (query_disease == pos_disease):
            continue
    else:
        found_drug = df_drug_sim[(df_drug_sim.DrugBank_ID1 == query_drug) & (df_drug_sim.DrugBank_ID2 == pos_drug)]
        drug_cheSim = found_drug['CheSim'].values[0]
        drug_pheSim = found_drug['PheSim'].values[0]
    
        found_disease = df_disease_sim[(df_disease_sim.Disease_ID1 == query_disease) & (df_disease_sim.Disease_ID2 == pos_disease)]
        disease_pheSim = found_disease['PheSim'].values[0]
        disease_geSim = found_disease['GeSim'].values[0]
    
        drug_sim = [drug_cheSim, drug_pheSim]
        disease_sim = [disease_geSim, disease_pheSim]
        
        sim = [math.sqrt(x[0]*x[1]) for x in list(product(drug_sim, disease_sim))]
        temp_df = pd.DataFrame(data = [[query_drug, query_disease, query_asso, sim[0], sim[1], sim[2], sim[3]]],
                           columns=['drug', 'disease', 'association', 'DrChe_DiGe', 'DrChe_DiPhe', 'DrPhe_DiGe', 'DrPhe_DiPhe'])
        descriptor = descriptor.append(temp_df)
        cnt = cnt +1
        if cnt % 10000 == 0:
            print "{} : ".format(cnt) + time.strftime('%a %H:%M:%S')
print 'end : ' + time.strftime('%a %H:%M:%S')
print cnt
print descriptor.shape

real_desc = descriptor.groupby(['drug','disease']).max()
real_desc.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_pos_descriptor_{}_{}.tsv".format(start, end-1),
                  sep = '\t', index=False)





1397
start : Tue 11:36:03
end : Tue 11:36:04
start : Tue 11:36:04
10000 : Tue 11:40:54
20000 : Tue 11:44:39
30000 : Tue 11:48:32
40000 : Tue 11:52:25
50000 : Tue 11:56:26
60000 : Tue 12:00:33
70000 : Tue 12:05:04
80000 : Tue 12:09:56
90000 : Tue 12:14:55
100000 : Tue 12:19:53
110000 : Tue 12:25:01
120000 : Tue 12:30:14
130000 : Tue 12:35:28
140000 : Tue 12:40:29
150000 : Tue 12:46:00
160000 : Tue 12:51:53
170000 : Tue 12:57:45
180000 : Tue 13:03:37
190000 : Tue 13:09:29
200000 : Tue 13:15:43
210000 : Tue 13:22:20
220000 : Tue 13:28:26
230000 : Tue 13:34:50
240000 : Tue 13:41:40
end : Tue 13:48:09
249821
(249821, 7)


In [4]:
descriptor.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/5_PREDICT_descriptor/test_pos_bef_descriptor_{}_{}.tsv".format(start, end-1),
                  sep = '\t', index=False)